In [1]:
import sys
sys.path.insert(0, "..")
import snscrape.modules.twitter as sntwitter
from fastapi.encoders import jsonable_encoder
import json
import pandas as pd
import numpy as np
from pymongo import MongoClient
import datetime
import re
from config import conf
from utils import translate
from cleaner import Cleaner

In [2]:
client = MongoClient(conf['mongo_uri'])
db = client.politweet

In [3]:
tweets = list(db.tweets.find({}, {'content':1}).limit(100))
corpus = []
for tweet in tweets:
    corpus.append(tweet['content'])
corpus = ' '.join(corpus)

In [8]:
df = pd.DataFrame(tweets)
df

,_id,content
0,612323fc5759170ef4695246,Je fais le voeu que les enseignements de ces e...
1,612323fc5759170ef4695247,Les récentes intempéries ont durement frappé n...
2,612323fc5759170ef4695248,Aux élections régionales qui auront lieu diman...
3,612323fc5759170ef4695249,Vote du PJL Climat @AssembleeNat : il faut mis...
4,612323fc5759170ef469524a,C’est avec plaisir que j’ai accueilli @franckr...
...,...,...
95,6123246a5759170ef46952a5,"@FrancoisManoury Non, mais je m’en souviens tr..."
96,6123246a5759170ef46952a6,Le patron du Raid raconte l'exfiltration de l'...
97,6123246a5759170ef46952a7,Une grande et belle page de l’histoire @HAC_F...
98,6123246a5759170ef46952a8,#lh merci @ArthurG_H de porter les couleurs #...


In [ ]:
def clean(txt):
    from bs4 import BeautifulSoup #Nettoyage d'HTML
    import nltk
    import unicodedata

    txt = BeautifulSoup(txt,"html.parser",from_encoding='utf-8').get_text()
    txt = txt.lower()
    txt = unicodedata.normalize('NFD', txt).encode('ascii', 'ignore').decode("utf-8")
    txt = re.sub('[^a-z_]', ' ', txt)
    french_stopwords = nltk.corpus.stopwords.words('french')
    stopwords = [unicodedata.normalize('NFD', sw).encode('ascii', 'ignore').decode("utf-8") for sw in french_stopwords]
    tokens = [w for w in txt.split() if (w not in stopwords)]
    stemmer=nltk.stem.SnowballStemmer('french')
    tokens_stem = [stemmer.stem(token) for token in tokens]
    return " ".join(tokens_stem)

df['content'] = df.content.apply(lambda x: clean(x))
   


In [9]:
cleaner = Cleaner(df)
soft_pipeline = ["emoji", "numbers", "dates", "non_asci", "fr_stopwords", "multiple_spaces"]
antitag_pipeline = ["userhandles", "hashtags"] + soft_pipeline
stemming_pipeline = ["emoji", "special_characters", "numbers", "fr_stopwords", "stemming_fr", "multiple_spaces"]
cleaner.clean(soft_pipeline)
cleaner.input

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vivien\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vivien\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\vivien\AppData\Local\Programs\Python\Python39\lib\site-packages\bs4\__init__.py:221: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


['lowercase', 'fix_escape', 'html_tags', 'urls', 'emails', 'multiple_spaces', 'emoji', 'numbers', 'dates', 'non_asci', 'fr_stopwords', 'multiple_spaces']


,_id,content
0,612323fc5759170ef4695246,voeu enseignements experiences douloureuses pe...
1,612323fc5759170ef4695247,recentes intemperies durement frappe voisins ....
2,612323fc5759170ef4695248,"elections regionales lieu dimanche juin, #jevo..."
3,612323fc5759170ef4695249,vote pjl climat @assembleenat : faut miser d'e...
4,612323fc5759170ef469524a,cest plaisir jai accueilli @franckriester damb...
...,...,...
95,6123246a5759170ef46952a5,"@francoismanoury non, men souviens jetais trib..."
96,6123246a5759170ef46952a6,patron raid raconte l'exfiltration l'ambassade...
97,6123246a5759170ef46952a7,grande belle page lhistoire @hac_foot tourne d...
98,6123246a5759170ef46952a8,#lh @arthurg_h porter couleurs #havre #axeseine
